In [13]:
# Import deepos
import deepops as dp

# Give your tensor a cool name.

In [14]:
# make Tensors
a = dp.Tensor([1.0]*100000, name = "leo")
b = dp.Tensor([2.0]*100000, name = "brad pitt")

In [15]:
print(a.name)
print(b.name)

leo
brad pitt


# Forward Operations.

In [16]:
# Tensor Operations on CPU
print("Add:", a+b, "Mul:", a*b)

Add: dp.Tensor  shape: (100000,), numpy: ([3. 3. 3. ... 3. 3. 3.], dtype=float32), cuda device: cpu Mul: dp.Tensor  shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), cuda device: cpu


In [17]:
# Default Device is cpu
a

dp.Tensor name: leo,  shape: (100000,), numpy: ([1. 1. 1. ... 1. 1. 1.], dtype=float32), cuda device: cpu

In [18]:
# throw these Tensors in our precious GPU
a.device('gpu') # i.e gpu:0
b.device('gpu')

In [19]:
# add them on GPU
addition = a.add(b)

In [20]:
# print the Addition
addition

dp.Tensor  shape: (100000,), numpy: ([3. 3. 3. ... 3. 3. 3.], dtype=float32), cuda device: cpu

In [21]:
# adds beautifully with itself!
a.add(a)

dp.Tensor  shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), cuda device: cpu

In [22]:
print(a + a)

dp.Tensor  shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), cuda device: cpu


In [23]:
print(a * b)

dp.Tensor  shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), cuda device: cpu


In [24]:
repr(a)

'dp.Tensor name: leo,  shape: (100000,), numpy: ([1. 1. 1. ... 1. 1. 1.], dtype=float32), cuda device: cpu'

# BackPropogration is real now.

In [ ]:
Tensor = dp.Tensor
a1 = Tensor([1.0, 3.0, 1.0])
b1 = Tensor([7.0, 3.0, 5.0])

a2 = Tensor([4.0, 3.0, 1.0])
a3 = Tensor([3.0, 3.0, 1.0])
a4 = Tensor([7.0, 1.0, 6.0])
b2 = Tensor([1.0, 21.0, 12.0])

c = a1 * b1 + a3
d = a2 * b2 + a4
out = c * d

In [26]:
# calculate gradients in style!!
out.backward()

1

In [27]:
# output gradients
out.grad

1

In [28]:
# Gradients
_ = [print(f"Gradients for {tensor} tensor is : {tensor.grad}") for tensor in [a1,a2,a3,a4,b1,b2,c,d]]

Gradients for dp.Tensor  shape: (3,), numpy: ([1. 3. 1.], dtype=float32), cuda device: cpu tensor is : [ 77. 192.  90.]
Gradients for dp.Tensor  shape: (3,), numpy: ([4. 3. 1.], dtype=float32), cuda device: cpu tensor is : [ 10. 252.  72.]
Gradients for dp.Tensor  shape: (3,), numpy: ([3. 3. 1.], dtype=float32), cuda device: cpu tensor is : [11. 64. 18.]
Gradients for dp.Tensor  shape: (3,), numpy: ([7. 1. 6.], dtype=float32), cuda device: cpu tensor is : [10. 12.  6.]
Gradients for dp.Tensor  shape: (3,), numpy: ([7. 3. 5.], dtype=float32), cuda device: cpu tensor is : [ 11. 192.  18.]
Gradients for dp.Tensor  shape: (3,), numpy: ([ 1. 21. 12.], dtype=float32), cuda device: cpu tensor is : [40. 36.  6.]
Gradients for dp.Tensor  shape: (3,), numpy: ([10. 12.  6.], dtype=float32), cuda device: cpu tensor is : [11. 64. 18.]
Gradients for dp.Tensor  shape: (3,), numpy: ([11. 64. 18.], dtype=float32), cuda device: cpu tensor is : [10. 12.  6.]


# Neural Network is REAL!!.

# Lets Train our Stupid Little Model on some Stupid Little data :P

In [30]:
import numpy as np

import deepops as dp
from deepops.model import Model


class SeqFC(Model):
    def __init__(self):
        super().__init__()
        self.dense1 = dp.layers.Dense(2, 2, activation="relu", name="dense1")
        self.dense2 = dp.layers.Dense(2, 1, name="dense2")

    def forward(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x


def loss(y, ypred):
    # MSE
    _loss = [(yb - ypb) * (yb - ypb) for yb, ypb in zip(y, ypred)]
    return sum(_loss) * dp.Tensor(1 / len(yb))


sequential = SeqFC()
batch_size = 10


for k in range(1000):
    xb = np.random.uniform(0, 10, (10, 2))
    yb = np.sum(xb, axis=-1)
    xb, yb = np.asarray(xb, np.float32), np.asarray(yb, np.float32)
    xb = [list(map(dp.Tensor, x)) for x in xb]
    # forward
    y_pred_b = list(map(sequential.forward, xb))
    yb = [dp.Tensor(y) for y in yb]
    total_loss = loss(yb, y_pred_b)
    # backward
    sequential.init_backward()
    total_loss.backward()
    # mini optimizer
    learning_rate = 0.03
    for p in sequential.parameters():
        p._data -= learning_rate * p.grad
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}")

step 0 loss 83.62093353271484
step 1 loss 153.6471710205078
step 2 loss 118.10765838623047
step 3 loss 164.33868408203125
step 4 loss 128.4536590576172
step 5 loss 64.83544158935547
step 6 loss 49.537322998046875
step 7 loss 33.880615234375
step 8 loss 16.972003936767578
step 9 loss 3.3445241451263428
step 10 loss 22.939016342163086
step 11 loss 11.186174392700195
step 12 loss 3.0199997425079346
step 13 loss 6.407691478729248
step 14 loss 4.100939750671387
step 15 loss 4.125068664550781
step 16 loss 4.403141498565674
step 17 loss 3.2661538124084473
step 18 loss 6.544332981109619
step 19 loss 2.8653032779693604
step 20 loss 9.085537910461426
step 21 loss 2.4606387615203857
step 22 loss 1.139976978302002
step 23 loss 1.7854589223861694
step 24 loss 1.2972829341888428
step 25 loss 2.2144083976745605
step 26 loss 1.8525009155273438
step 27 loss 2.2874062061309814
step 28 loss 1.6249809265136719
step 29 loss 1.6843258142471313
step 30 loss 0.5546727180480957
step 31 loss 0.5272324681282043


step 367 loss 0.03391159698367119
step 368 loss 0.013019976206123829
step 369 loss 0.014645658433437347
step 370 loss 0.03332337364554405
step 371 loss 0.03386068716645241
step 372 loss 0.0099871177226305
step 373 loss 0.013534343801438808
step 374 loss 0.017259616404771805
step 375 loss 0.00842084176838398
step 376 loss 0.02906404249370098
step 377 loss 0.0176133643835783
step 378 loss 0.021020179614424706
step 379 loss 0.016055211424827576
step 380 loss 0.025675857439637184
step 381 loss 0.00948378723114729
step 382 loss 0.021924257278442383
step 383 loss 0.051244255155324936
step 384 loss 0.018372265622019768
step 385 loss 0.02140783704817295
step 386 loss 0.019434157758951187
step 387 loss 0.020880667492747307
step 388 loss 0.01746256649494171
step 389 loss 0.023547757416963577
step 390 loss 0.015852095559239388
step 391 loss 0.016302192583680153
step 392 loss 0.010566054843366146
step 393 loss 0.013234619982540607
step 394 loss 0.010825223289430141
step 395 loss 0.0160101111978292

step 727 loss 0.007343857083469629
step 728 loss 0.01037329901009798
step 729 loss 0.005526815541088581
step 730 loss 0.009027359075844288
step 731 loss 0.014924361370503902
step 732 loss 0.006613367702811956
step 733 loss 0.010621733032166958
step 734 loss 0.00795858632773161
step 735 loss 0.012141814455389977
step 736 loss 0.010093051008880138
step 737 loss 0.01205157395452261
step 738 loss 0.007206119131296873
step 739 loss 0.004646790213882923
step 740 loss 0.00639472296461463
step 741 loss 0.005433785263448954
step 742 loss 0.006503518670797348
step 743 loss 0.007213833276182413
step 744 loss 0.013159990310668945
step 745 loss 0.005820981692522764
step 746 loss 0.006920437794178724
step 747 loss 0.0023100438993424177
step 748 loss 0.008156869560480118
step 749 loss 0.01049779076129198
step 750 loss 0.010280916467308998
step 751 loss 0.00906248576939106
step 752 loss 0.012461538426578045
step 753 loss 0.003510805545374751
step 754 loss 0.01672191172838211
step 755 loss 0.0082415407

# Lets infere this...

In [31]:
print(sequential.forward([dp.Tensor(1.0), dp.Tensor(1.0)]))

dp.Tensor  shape: (), numpy: (2.1475282, dtype=float32), cuda device: cpu
